In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java (Windows)
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-29 05:06:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-11-29 05:06:16 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
# Start Spark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Toys").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("header",True) \
                    .option("sep", "\t") \
                    .option("multiLine", True) \
                    .option("quote","\"") \
                    .option("escape","\"") \
                    .option("ignoreTrailingWhiteSpace", True) \
                    .csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"))

In [7]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [8]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [10]:
# Count the number of records (rows) in the dataset.
print(f"Number of records in the dataset is {df.count()}.")

Number of records in the dataset is 4859927.


In [11]:
#drop duplicates and incomplete rows
df = df.dropna()
df = df.dropDuplicates()
#show total counts of reviews after dropping duplicates and incomplete rows
print(df.count())

4859184


In [12]:
# Transform the dataset to fit the tables in the schema file.
vine_reviews = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1002PTVVPI7IP|          5|            1|          1|   N|
|R1005AJH0IN031|          3|            0|          0|   N|
|R100FLYOCRSS0E|          5|            0|          0|   N|
|R100Z6264Z6B7C|          5|            0|          0|   N|
|R1012HPPU8SM22|          4|            1|          1|   N|
|R1017RTVKF6W9N|          1|            0|          1|   N|
|R101EEDKP57GEK|          1|            0|          1|   N|
|R101GBYOCYPJRO|          5|            0|          0|   N|
|R101H5IJ2X94SK|          5|            2|          2|   N|
|R101H629AQF9CK|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [13]:
#select vine reviews
from pyspark.sql.functions import col
vine_reviews_yes = vine_reviews.filter(col("vine")=="Y")
#show the total count of vine reviews
print(vine_reviews_yes.count())
vine_reviews_yes.show(10)

41811
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RUB4AUWGHG16G|          4|           25|         27|   Y|
|R3LO5CH624HA1S|          5|            2|          3|   Y|
| RMJEGES0TVU0D|          2|            2|          4|   Y|
|R3AZRHJ052HJBX|          4|            1|          2|   Y|
|R3VTLTP8EHFI5A|          5|            0|          1|   Y|
| RNIX85WIG686Y|          5|            1|          2|   Y|
|R1MY99OEYLQDG0|          5|            0|          0|   Y|
|R2J0M5H0D36EPZ|          3|            0|          1|   Y|
|R3HC5N32EE0R2U|          5|            0|          0|   Y|
|R3ELON0ZBNEXL5|          5|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [14]:
vine_reviews_yes.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)



In [15]:
# Load the DataFrames that correspond to tables into an RDS instance. 
#endpoint and setup a new database in postgres aws cloud database
db_url = "jdbc:postgresql://<AWS RDS Endpoint>/<database name>"
config = {
    "user":"<user name>",
    "password":"<password>",
    "driver":"org.postgresql.Driver"
}

In [16]:
vine_reviews_yes.write.jdbc(url=db_url,table="toys_review",mode="append",properties=config)